In [8]:
from arcgis.features import FeatureLayer
import os
import requests
import time

# Access the feature layer
feature_layer_url = "https://services.arcgis.com/0jRlQ17Qmni5zEMr/arcgis/rest/services/lpt_points_view/FeatureServer/0"
layer = FeatureLayer(feature_layer_url)

# Query the feature layer to get all features
features = layer.query(where="1=1", out_fields="*").features

# Directory to save the photos
save_dir = "downloaded_photos"
os.makedirs(save_dir, exist_ok=True)

# Function to download and save attachment
def download_attachment(feature, attachment_id, attachment_name):
    attachment_url = f"{feature_layer_url}/{feature.attributes['OBJECTID']}/attachments/{attachment_id}"
    response = requests.get(attachment_url, stream=True)
    if response.status_code == 200:
        file_path = os.path.join(save_dir, attachment_name)
        with open(file_path, 'wb') as file:
            for chunk in response.iter_content(chunk_size=128):
                file.write(chunk)
        print(f"Downloaded: {attachment_name}")
    else:
        print(f"Failed to download: {attachment_name}")

# Loop through each feature to download the photo
for feature in features:
    attributes = feature.attributes
    attachment_info = None

    # Retry logic for fetching attachment info
    retries = 3
    for attempt in range(retries):
        try:
            attachment_info = layer.attachments.get_list(feature.attributes['OBJECTID'])
            break
        except Exception as e:
            print(f"Error fetching attachments: {e}")
            if "503" in str(e):
                print("Service unavailable. Retrying...")
                time.sleep(5)  # Wait for 5 seconds before retrying
            else:
                raise
    else:
        print(f"Failed to fetch attachments after {retries} attempts. Skipping feature {feature.attributes['OBJECTID']}.")
        continue

    for attachment in attachment_info:
        attachment_id = attachment['id']
        # Assuming the feature has fields named 'Parcel', 'TRANSECT', and 'BEARING'
        attachment_name = f"{attributes['Parcel']}_{attributes['TRANSECT']}_{attributes['BEARING']}.jpg"
        
        # Check if the file already exists
        file_path = os.path.join(save_dir, attachment_name)
        if not os.path.exists(file_path):
            download_attachment(feature, attachment_id, attachment_name)
        else:
            print(f"Skipped (already exists): {attachment_name}")

print("Download completed.")


Skipped (already exists): IND086_7_200.jpg
Skipped (already exists): IND086_13_206.jpg
Skipped (already exists): IND086_3_304.jpg
Skipped (already exists): IND086_1_312.jpg
Skipped (already exists): IND086_15_316.jpg
Skipped (already exists): IND086_17_336.jpg
Skipped (already exists): IND086_5_51.jpg
Skipped (already exists): IND086_9_61.jpg
Skipped (already exists): IND086_11_80.jpg
Skipped (already exists): MAN038_23_2.jpg
Skipped (already exists): MAN038_17_265.jpg
Skipped (already exists): MAN038_19_313.jpg
Skipped (already exists): MAN038_21_324.jpg
Skipped (already exists): MAN038_15_115.jpg
Skipped (already exists): MAN038_13_129.jpg
Skipped (already exists): MAN038_7_166.jpg
Skipped (already exists): MAN038_11_255.jpg
Skipped (already exists): MAN038_1_291.jpg
Skipped (already exists): MAN038_3_292.jpg
Skipped (already exists): MAN038_9_45.jpg
Skipped (already exists): MAN038_5_46.jpg
Skipped (already exists): LNP019_11_7.jpg
Skipped (already exists): LNP019_5_51.jpg
Skipped (

In [2]:
from arcgis.features import FeatureLayer
import os
import requests


C:\Users\znelson\AppData\Roaming\Python\Python39\site-packages\pandas\core\computation\expressions.py:21: UserWarning: Pandas requires version '2.8.4' or newer of 'numexpr' (version '2.8.1' currently installed).
  from pandas.core.computation.check import NUMEXPR_INSTALLED
C:\Users\znelson\AppData\Roaming\Python\Python39\site-packages\pandas\core\arrays\masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.4' currently installed).
  from pandas.core import (


In [3]:

# Access the feature layer
feature_layer_url = "https://services.arcgis.com/0jRlQ17Qmni5zEMr/arcgis/rest/services/lpt_points_view/FeatureServer/0"
layer = FeatureLayer(feature_layer_url)


In [4]:

# Query the feature layer to get all features
features = layer.query(where="1=1", out_fields="*").features


In [5]:

# Directory to save the photos
save_dir = "downloaded_photos"
os.makedirs(save_dir, exist_ok=True)


In [6]:

# Loop through each feature to download the photo
for feature in features:
    attributes = feature.attributes
    attachment_info = layer.attachments.get_list(feature.attributes['OBJECTID'])

    for attachment in attachment_info:
        attachment_id = attachment['id']
        # Assuming the feature has fields named 'parcel_id', 'transect_id', and 'bearing_id'
        attachment_name = f"{attributes['Parcel']}_{attributes['TRANSECT']}_{attributes['BEARING']}.jpg"
        attachment_url = f"{feature_layer_url}/{feature.attributes['OBJECTID']}/attachments/{attachment_id}"

        response = requests.get(attachment_url, stream=True)
        if response.status_code == 200:
            file_path = os.path.join(save_dir, attachment_name)
            with open(file_path, 'wb') as file:
                for chunk in response.iter_content(chunk_size=128):
                    file.write(chunk)
            print(f"Downloaded: {attachment_name}")
        else:
            print(f"Failed to download: {attachment_name}")

print("Download completed.")



Downloaded: IND086_7_200.jpg
Downloaded: IND086_13_206.jpg
Downloaded: IND086_3_304.jpg
Downloaded: IND086_1_312.jpg
Downloaded: IND086_15_316.jpg
Downloaded: IND086_17_336.jpg
Downloaded: IND086_5_51.jpg
Downloaded: IND086_9_61.jpg
Downloaded: IND086_11_80.jpg
Downloaded: MAN038_23_2.jpg
Downloaded: MAN038_17_265.jpg
Downloaded: MAN038_19_313.jpg
Downloaded: MAN038_21_324.jpg
Downloaded: MAN038_15_115.jpg
Downloaded: MAN038_13_129.jpg
Downloaded: MAN038_7_166.jpg
Downloaded: MAN038_11_255.jpg
Downloaded: MAN038_1_291.jpg
Downloaded: MAN038_3_292.jpg
Downloaded: MAN038_9_45.jpg
Downloaded: MAN038_5_46.jpg
Downloaded: LNP019_11_7.jpg
Downloaded: LNP019_5_51.jpg
Downloaded: LNP019_7_125.jpg
Downloaded: LNP019_3_165.jpg
Downloaded: LNP019_9_228.jpg
Downloaded: LNP019_13_257.jpg
Downloaded: LNP019_15_283.jpg
Downloaded: LNP019_1_220.jpg
Downloaded: UNW031_11_39.jpg
Downloaded: UNW031_5_253.jpg
Downloaded: UNW031_9_102.jpg
Downloaded: UNW031_15_170.jpg
Downloaded: UNW031_13_160.jpg
Download

Exception: An error occurred.
(Error Code: 503)